*Debéis decidir qué problema queréis abordar lo antes posible y comunicar vuestra elección (por correo electrónico) enviando la evaluación preliminar del problema a bejar@cs.upc.edu no más tarde de 24 de Octubre. Indicad también todoslos nombres de los miembros del grupo.*

# **Informe Previ**

---

Juli Sahun Montejano

Victor Pla Sanchis

### Descripció del problema


---
Hem decidit abarcar un problema de classificació binària. Donat un conjunt de dades, estudiar-les i analitzar-les per poder aplicar un model predictor de classificació que ens separi dues classes significativament variades.

### Descripció del dataset


---
El dataset ha de complir:

El dataset que hem escollit recull informacio sobre carteres de ethereum on algunes de les cuals son fraudulentes, el nostre objectiu sera entrenar un model que pugui predir basant-se amb la resta de dades, quins perfils son potencialment fraudulents. Acountinuacio, enumerem els camps del nostre dataset:

* address : string
* flag : boolean
* min/max/avgTimeBetweeSentTnx : float
* min/max/avgTimeBetweeRecTnx : float
* lifetime : float
* sentTransactions : float
* receivedTransactions : float
* createdContracts : float
* numUniqSentAddress : float
* min/max/avgValReceived : float
* totalTransactions : float
* totalEtherSent : float
* totalEtherReceived : float
* totalEtherSentContracts : float
* totalEtherBalance : float
* activityDays : float
* dailyMax : float
* ratioRecSent : float
* ratioSentTotal : float
* ratioRecTotal : float
* giniSent : float
* giniRec : float
* txFreq : float
* stdBalanceEth : float



In [ ]:
# !pip3 install pandas==1.5.1 --user --quiet
# !pip3 install numpy==1.23.4 --user --quiet
# !pip3 install scikit-learn==1.2.0 --user --quiet
# !pip3 install yellowbrick==1.5 --user --quiet
# !pip3 install scikit-optimize==0.9.0 --user --quiet
# !pip3 install matplotlib==3.6.0 --user --quiet
# !pip3 install lime

In [ ]:
import pandas as pd
import numpy as np
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import sklearn
import yellowbrick
import skopt
import lime

In [ ]:
print(f'Pandas version: {pd.__version__}')
print(f'Numpy version: {np.__version__}')
print(f'Scikit-learn version: {sklearn.__version__}')
print(f'Yellowbrick version: {yellowbrick.__version__}')
print(f'Scikit-optimize version: {skopt.__version__}')
print(f'Matplotlib version: {mpl.__version__}')

In [ ]:
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import  ConfusionMatrixDisplay,\
                  classification_report,  RocCurveDisplay, \
                    accuracy_score, f1_score, precision_score, recall_score

import lime.lime_tabular
from sklearn.model_selection import GridSearchCV
from dtreeviz.trees import dtreeviz


from yellowbrick.target.feature_correlation import feature_correlation
from sklearn.decomposition import PCA
from IPython.display import display, HTML
show_html = lambda html: display(HTML(html))
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.precision', 3)

In [ ]:
def save_results(clf, X_test, y_test, nclf, df):
    df.loc[nclf,'test acc'] = accuracy_score(y_test, clf.predict(X_test))
    df.loc[nclf,'precision score (Fraudulent)'] = precision_score(y_test, clf.predict(X_test), average='binary')
    df.loc[nclf,'recall score (Fraudulent)'] = recall_score(y_test, clf.predict(X_test), average='binary')
    df.loc[nclf,'f1 score (Fraudulent)'] = f1_score(y_test, clf.predict(X_test), average='binary')
    return df

def displayConfusionMatrixAndRocCurve(estimator, X, y):
    fig, axes = plt.subplots(1,2,figsize=(15,5))
    ax = axes.reshape(-1)[0]
    ConfusionMatrixDisplay.from_estimator(estimator, X, y, display_labels=target_names, ax=ax)
    ax = axes.reshape(-1)[1]
    RocCurveDisplay.from_estimator(estimator, X, y, ax=ax, pos_label=1)
    plt.tight_layout()

def weights(estimator):
    coefs = pd.DataFrame(estimator.best_estimator_.coef_, columns=X_train.columns)
    coefs.columns = X_train.columns
    plt.figure(figsize=(25,2))
    sns.heatmap(coefs.abs().round(), annot=True, cbar=True, cmap='Blues', linewidths=.5, annot_kws={"size": 12}, fmt='g')

def explainer(estimator, X, y, i = 0):
    if (type(X) == pd.core.frame.DataFrame):
        X = X.to_numpy()        
    model_explainer = lime.lime_tabular.LimeTabularExplainer(X, feature_names=list(data.columns[:-1]),class_names=target_names, verbose=True, mode='classification')
    exp = model_explainer.explain_instance(X[i], estimator.predict_proba, num_features=6)
    exp.show_in_notebook(show_table=True)

results_df_lineal = pd.DataFrame()
results_df_no_lineal = pd.DataFrame()
target_names=['Normal', 'Fraudulent']
niter = 15
cv = 5

In [ ]:
import pickle
import glob
import os

def save_state(variables):
    for name, value in variables.items():
        with open(f'store/{name}.pickle', 'wb') as f:
            pickle.dump(value, f)
def load_state():
    variables = {}
    for filename in glob.iglob('store/*'):
        with open(filename, 'rb') as f:
            name = filename[6:].replace('.pickle', '')
            variables[name] = pickle.load(f)
    globals().update(variables)

def clear_state():
    for filename in glob.iglob('store/*'):
        os.remove(filename)
    

In [40]:
# vars = {
#     'results_df_lineal': results_df_lineal, 
#     'results_df_no_lineal': results_df_no_lineal,
#     'data': data,
#     'X_train': X_train,
#     'X_test': X_test,
#     'y_train': y_train,
#     'y_test': y_test,
#     'X_train_n': X_train_n,
#     'X_test_n': X_test_n,
#     'lda': lda,
#     'lsvc_gs': lsvc_gs,
#     'knn_gs': knn_gs,
#     'svc_gs': svc_gs,
#     'mlp_gs': mlp_gs,
#     'rf_bs': rf_bs
#     }
# save_state(vars)

In [ ]:
load_state()

### Descàrrega del dataset

---

In [ ]:
#https://www.kaggle.com/datasets/gescobero/ethereum-fraud-dataset
data = pd.read_csv("./data/eth_illicit_features.csv")
print("Filas:",data.shape[0])
print("Columnas:",data.shape[1])

### Analisis del dataset

---

In [ ]:
data.head()

Eliminem la columna $address$ ja que no ens aporta cap informacio

In [ ]:
data.drop(columns=['address'], inplace=True)
data.describe(include='all').T

Separem les dades en el conjunt predictor i el conjunt objectiu. Aprofitem tambe per normalitzar les dades, ja que tenen rangs de valors molt diferents

In [ ]:
X= data.iloc[:,1:]
y= data.loc[:,'flag'].copy()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_train_n = scaler.fit_transform(X_train)
X_test_n = scaler.transform(X_test)

Podem representar la distribucio de les variables per veure si alguna segueix algun comportament especific:

In [ ]:
''' aixo esta comentat perque triga molt en execurar-se, pero es una manera de veure la distribucio de les dades'''

# #ejecutar con matplotlib 3.6.0!! https://github.com/mwaskom/seaborn/issues/3072

# fig, axes = plt.subplots(14,2,figsize=(15,50))
# X_train_nr = X_train_n.round(3)
# X_test_nr = X_test_n.round(3)
# X_train_frame = pd.DataFrame(X_train_nr)
# X_train_frame.columns=X.columns
# for i, c in enumerate(X.columns):
#     ax = axes.reshape(-1)[i]
#     if X_train_frame[c].dtype.kind == 'O':
#         a = sns.countplot(x=c,data=X_train_frame,ax=ax)
#     else:
#         b = sns.histplot(x=c,data=X_train_frame,ax=ax)
# plt.tight_layout()

Podem representar en un _Heatmap_ la correlacio entre les variables

In [ ]:
corr = X_train.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
plt.subplots(figsize=(10, 8))
sns.heatmap(corr, mask=mask, cmap='seismic',  center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5});

En Aquest grafic podem veure la relacio de cada una de les variables amb la variable objectiu, en aquest cas, $flag$

In [ ]:
plt.figure(figsize=(10,8))
visualizer = feature_correlation(X_train, y_train, labels=list(X_train.columns),method='mutual_info-classification');

In [ ]:
pca = PCA().fit(X_train_n)

Calculant el PCA ja ens donem compte que no sera una feina facil, no es fins prop de la 15 component, on podem explicar un 80% de la variancia de les dades

In [ ]:
fig = plt.figure(figsize=(8,6));
plt.plot(range(1,len(pca.explained_variance_ratio_ )+1),pca.explained_variance_ratio_ ,alpha=0.8,marker='.',label="Variancia Explicada");
y_label = plt.ylabel('Variancia explicada');
x_label = plt.xlabel('Componentes');
plt.plot(range(1,len(pca.explained_variance_ratio_ )+1),
         np.cumsum(pca.explained_variance_ratio_),
         c='red',marker='.',
         label="Variancia explicada acumulativa");
plt.legend();
plt.title('Porcentaje de variancia explicada por componente');

Les dues primeres components del PCA expliquen la misera variança del 25-30%. Aixi doncs, es d'esperar que no poguem visualitzar cap tipus de resultat satisfactori

In [ ]:
X_trans = pca.transform(X_train)
plt.figure(figsize=(8,8));
sns.scatterplot(x=X_trans[:,0], y=X_trans[:,1], hue=y_train)

### Models linials

#### LDA


Probem de clasificar les dades fent servir un model d'Analisis de Discriminacio Lineal

Obtenim el resultat de la mitjana de les 10 validacions encreuades

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
print(np.mean(cross_val_score(lda,X_train_n,y_train,cv=10)))

Com podem veure, el resultat obtinugt esta lluny de ser bo, pero no es tant dolent com en un principi ens podriem esperar/

Podem veure el _clasification report_ del model par veure en mes detall els seus resultats:

In [ ]:
lda.fit(X_train, y_train)
print(classification_report(lda.predict(X_test_n), y_test,target_names=target_names))
results_df_lineal = save_results(lda, X_test_n, y_test, 'LDA', results_df_lineal)

Mirant la matriu de confusio veiem que, aproximadament, un 20% dels casos fraudulents els classifiquem com a normals, i un 20% dels casos normals els classifiquem com a fraudulents

Podem veure la curva ROC per veure en mes detall quina proporcio de casos fraudulents clasifiquem com a tal, en relacio els casos totals que clasifiquem com a fraudulents.

Com podem veure, es necessari prop d'un 60% de falsos positius per tenir un resultat acceptable de vertaders positius

In [ ]:
displayConfusionMatrixAndRocCurve(lda, X_test, y_test)

In [ ]:
coefs = pd.DataFrame(lda.coef_, columns=X.columns)
coefs.columns = X.columns
plt.figure(figsize=(20,2))
sns.heatmap(coefs.abs().round(), annot=True, cbar=True, cmap='Blues', linewidths=.5, annot_kws={"size": 12})

In [ ]:
explainer(lda, X_test, y_test, 5)

#### Lineal SVC

Podem provar un model de regresio logistica que no assumeix cap tipus de distribucio en les dades 

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
lsvc = LinearSVC()
print(np.mean(cross_val_score(lsvc,X_train_n,y_train,cv=10)))

In [ ]:
param = {'C':10**np.linspace(-3,3,101), 
              'penalty':['l1','l2'], 
              'loss':['hinge','squared_hinge']
            }
lsvc = LinearSVC(max_iter=25000)
lsvc_gs =  GridSearchCV(lsvc,param,cv=cv, n_jobs=-1, refit=True)
lsvc_gs.fit(X_train_n, y_train);
show_html(pd.DataFrame(lsvc_gs.cv_results_).loc[:,['params','mean_test_score','rank_test_score']].sort_values(by='rank_test_score').head().to_html())
print(classification_report(lsvc_gs.predict(X_test_n),y_test,target_names=target_names))

In [ ]:
results_df_lineal = save_results(lsvc_gs, X_test_n, y_test, 'LSVC', results_df_lineal)

In [ ]:
displayConfusionMatrixAndRocCurve(lsvc_gs, X_test_n, y_test)

In [ ]:
weights(lsvc_gs)

In [ ]:
from sklearn.calibration import CalibratedClassifierCV
clf = CalibratedClassifierCV(lsvc_gs.best_estimator_) 
clf.fit(X_train_n, y_train)
explainer(clf, X_test_n, y_test)

#### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn =  KNeighborsClassifier()
print(np.mean(cross_val_score(knn,X_train_n,y_train,cv=10)))

In [ ]:
param = {'n_neighbors':[5, 10, 15, 20, 25, 30], 
          'weights':['uniform', 'distance'], 
          'leaf_size':[20, 30, 40, 50], #irrellevant
          'metric': ['l2', 'l1', 'cosine']}

knn_gs =  GridSearchCV(knn,param,cv=cv, n_jobs=-1)
knn_gs.fit(X_train_n, y_train);

In [ ]:
show_html(pd.DataFrame(knn_gs.cv_results_).loc[:,['params', 'mean_test_score','rank_test_score']].sort_values(by='rank_test_score').head().to_html())

In [ ]:
print(classification_report(knn_gs.predict(X_test_n), y_test,target_names=target_names))
results_df_lineal = save_results(knn_gs, X_test_n, y_test, 'KNN', results_df_lineal)

In [ ]:
displayConfusionMatrixAndRocCurve(knn_gs, X_test_n, y_test)

In [ ]:
explainer(knn_gs, X_test_n, y_test, 5)

In [ ]:
weights(knn_gs)

### Resultats

In [ ]:
results_df_lineal.sort_values(by=['test acc'], ascending=False)

### Metodes no linials

#### SVC

In [ ]:
from sklearn.svm import SVC

In [ ]:
param = {'C':10**np.linspace(-3,3,101)}

svc = SVC(max_iter=250000, random_state=0, probability=True)
svc_gs = GridSearchCV(svc,param, cv=cv, n_jobs=-1, refit=True)
svc_gs.fit(X_train_n, y_train)
show_html(pd.DataFrame(svc_gs.cv_results_).loc[:,['params', 'mean_test_score','rank_test_score']].sort_values(by='rank_test_score').head().to_html())

In [ ]:
print(classification_report(svc_gs.predict(X_test_n), y_test,target_names=target_names))
results_df_no_lineal = save_results(svc_gs, X_test_n, y_test, 'SVC', results_df_no_lineal)

In [ ]:
displayConfusionMatrixAndRocCurve(svc_gs, X_test_n, y_test)

In [ ]:
explainer(svc_gs, X_test, y_test, 5)

#### MLP

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

In [ ]:
mlp = MLPClassifier(max_iter=1000, early_stopping=True, random_state=10)
print(np.mean(cross_val_score(mlp,X_train_n,y_train,cv=10)))

param = {'hidden_layer_sizes':[1, 25, 50, 75, 100, 125, 150], 
         'activation':['logistic', 'relu', 'tanh', 'identity'], 
         'learning_rate_init': [0.0001, 0.001, 0.01, 0.1]  }

mlp =  MLPClassifier(max_iter=1000, early_stopping=True,random_state=10)
mlp_gs =  GridSearchCV(mlp,param,cv=cv, n_jobs=-1, refit=True)
mlp_gs.fit(X_train_n, y_train);
show_html(pd.DataFrame(mlp_gs.cv_results_).loc[:,['params', 'mean_test_score','rank_test_score']].sort_values(by='rank_test_score').head().to_html())

In [ ]:
print(classification_report(mlp_gs.predict(X_test_n), y_test,target_names=target_names))

In [ ]:
displayConfusionMatrixAndRocCurve(mlp_gs, X_test_n, y_test)

In [ ]:
explainer(mlp_gs, X_test_n, y_test, 5)

In [ ]:
print(classification_report(mlp_gs.predict(X_test_n), y_test,target_names=target_names))
results_df_no_lineal = save_results(mlp_gs, X_test_n, y_test, 'MPL', results_df_no_lineal)

#### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from skopt import BayesSearchCV
rf = RandomForestClassifier(random_state=0)
print(np.mean(cross_val_score(rf,X_train_n,y_train,cv=10)))

param = {'n_estimators':[10,100, 150, 200],
         'min_samples_split': [1, 2,4,32, 64],
         'min_samples_leaf': [1, 2,4,32],
         'max_features': [1,2,32, 64, 128]
         }

rf=RandomForestClassifier(warm_start=True, random_state=0)
rf_bs =  BayesSearchCV(rf,param,n_iter=15, cv=cv, n_jobs=-1, refit=True, random_state=0)
rf_bs.fit(X_train_n, y_train);
show_html(pd.DataFrame(rf_bs.cv_results_).loc[:,['params', 'mean_test_score','rank_test_score']].sort_values(by='rank_test_score').head().to_html())

In [ ]:
print(classification_report(rf_bs.predict(X_test_n), y_test,target_names=target_names))
results_df_no_lineal = save_results(rf_bs, X_test_n, y_test, 'RF', results_df_no_lineal)

In [ ]:
displayConfusionMatrixAndRocCurve(rf_bs, X_test_n, y_test)

In [ ]:
explainer(rf_bs, X_test_n, y_test, 5)

In [ ]:
results_df_no_lineal.sort_values(by=['test acc'], ascending=False)

In [ ]:
pd.concat([results_df_lineal, results_df_no_lineal], axis=0).sort_values(by=['test acc'], ascending=False)